In [1]:
from pathlib import Path
import os
import sys

from dotenv import load_dotenv


def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
    raise FileNotFoundError("Could not find repo root with app/ directory")


ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)
if os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = os.environ["OPENAI_API_KEY"].strip()

print("ROOT_DIR:", ROOT_DIR)
print("KEY LOADED:", bool(os.getenv("OPENAI_API_KEY")))


ROOT_DIR: /Users/user/Desktop/RAG/backend
KEY LOADED: True


In [2]:
from app.rag.pipeline import RAGConfig, run_rag
import json

normalize_keywords = True  # 표준화 키워드 사용 (라우터 매칭 기반)

query = input("질문 입력: ").strip()
if not query:
    raise ValueError("질문을 입력해 주세요.")

config = RAGConfig(top_k=4, normalize_keywords=normalize_keywords)
result = await run_rag(query, config=config)

print("던진 질문:", query)
print("route:", result.get("routing", {}).get("route"))
print(json.dumps({
    "currentSituation": result.get("currentSituation", []),
    "nextStep": result.get("nextStep", []),
    "guidanceScript": result.get("guidanceScript", "")
}, ensure_ascii=False, indent=2))


던진 질문: 다둥이 카드 발급
route: card_usage
{
  "currentSituation": [
    {
      "id": "guide_35",
      "title": "꼭 필요한 카드만 발급받으세요",
      "keywords": [
        "#서울시다둥이행복카드",
        "#발급"
      ],
      "content": "신용카드를 3개 이상 소지할 경우 신용등급이나 이용한도에 영향을 줄 수 있으니 꼭 필요한 카드만 발급받으시기 바랍니다.",
      "detailContent": "제목: 꼭 필요한 카드만 발급받으세요\n\n내용: 신용카드를 3개 이상 소지하실 경우 신용등급이나 카드 이용한도 등에 영향을 미칠 수 있으므로 반드시 필요한 카드만 발급받아 이용하시기 바랍니다.",
      "relevanceScore": 0.023151515151515152
    },
    {
      "id": "guide_34",
      "title": "가족간 합리적 소비를 원한다면 가족카드를 이용하세요",
      "keywords": [
        "#서울시다둥이행복카드",
        "#발급"
      ],
      "content": "가족카드는 본인회원의 신용으로 가족들이 발급받을 수 있으며, 결제와 명세서가 통합되어 합리적 소비가 가능합니다.",
      "detailContent": "제목: 가족간 합리적 소비를 원한다면 가족카드를 이용하세요\n\n내용: 가족카드는 본인회원의 신용으로 본인회원의 배우자와 부모, 자녀 등 가족들이 발급받을 수 있는 상품입니다. 가족카드는 하나의 계좌로 명세서 발송과 결제가 통합되므로 가족의 합리적 소비 계획이 가능하며, 가족카드 이용금액이 본인카드 이용금액에 합산되어 포인트가 적립되므로 보다 경제적입니다.",
      "relevanceScore": 0.020384615384615386
    }
  ],
  "nextStep": [
    {
   

In [3]:
'''import subprocess, sys

result = subprocess.run(
    [sys.executable, "app/rag/scripts/run_regression.py"],
    capture_output=True,
    text=True,
)
print(result.stdout)
print(result.stderr)'''


'import subprocess, sys\n\nresult = subprocess.run(\n    [sys.executable, "app/rag/scripts/run_regression.py"],\n    capture_output=True,\n    text=True,\n)\nprint(result.stdout)\nprint(result.stderr)'

In [4]:
'''import subprocess, sys
subprocess.run(
    [sys.executable, "app/rag/scripts/run_regression.py"],
    cwd="/Users/user/Desktop/RAG/backend",
)
'''

'import subprocess, sys\nsubprocess.run(\n    [sys.executable, "app/rag/scripts/run_regression.py"],\n    cwd="/Users/user/Desktop/RAG/backend",\n)\n'

In [5]:
from pathlib import Path
import os
import sys

import psycopg2
from dotenv import load_dotenv

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
    raise FileNotFoundError("Could not find repo root with app/ directory")

ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)

def db_config():
    host = os.getenv("DB_HOST_IP") or os.getenv("DB_HOST")
    cfg = {
        "host": host,
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": int(os.getenv("DB_PORT", "0")) or None,
    }
    missing = [k for k, v in cfg.items() if not v]
    if missing:
        raise ValueError(f"Missing DB settings: {missing}")
    return cfg

def fetch(sql, params=None):
    with psycopg2.connect(**db_config()) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params or ())
            return cur.fetchall()

def show_schema(table: str):
    rows = fetch(
        """
        SELECT column_name, data_type, udt_name
        FROM information_schema.columns
        WHERE table_name = %s
        ORDER BY ordinal_position;
        """,
        (table,),
    )
    return rows

def sample_row(table: str):
    rows = fetch(
        f"""
        SELECT id, metadata, LEFT(content, 200)
        FROM {table}
        ORDER BY id DESC
        LIMIT 1;
        """
    )
    return rows[0] if rows else None

print("ROOT_DIR:", ROOT_DIR)
print("card_tbl schema:", show_schema("card_tbl"))
print("guide_tbl schema:", show_schema("guide_tbl"))

card_sample = sample_row("card_tbl")
guide_sample = sample_row("guide_tbl")
print("card_tbl sample:", card_sample)
print("guide_tbl sample:", guide_sample)

# metadata keys 분포(상위)
def metadata_keys_count(table: str):
    rows = fetch(
        f"""
        SELECT key, COUNT(*) 
        FROM (
            SELECT jsonb_object_keys(metadata) AS key
            FROM {table}
        ) t
        GROUP BY key
        ORDER BY COUNT(*) DESC
        LIMIT 15;
        """
    )
    return rows

print("card_tbl metadata keys:", metadata_keys_count("card_tbl"))
print("guide_tbl metadata keys:", metadata_keys_count("guide_tbl"))


ROOT_DIR: /Users/user/Desktop/RAG/backend
card_tbl schema: [('id', 'integer', 'int4'), ('content', 'text', 'text'), ('metadata', 'jsonb', 'jsonb'), ('embedding', 'USER-DEFINED', 'vector')]
guide_tbl schema: [('id', 'integer', 'int4'), ('content', 'text', 'text'), ('metadata', 'jsonb', 'jsonb'), ('embedding', 'USER-DEFINED', 'vector')]
card_tbl sample: (228, {'id': 'card_171', 'title': '포인트 적립 제외', 'category1': 'N/A', 'category2': 'N/A'}, '제목: 포인트 적립 제외\n\n내용: 포인트 적립 제외 대상은 아래와 같습니다. - 단기카드대출(현금서비스), 장기카드대출(카드론), 연회비, 각종 수수료이자(할부수수료, SMS이용수수료, 연체이자 등), 오토리스 대금, 기프트카드선불카드 구매  충전금액, 상품권 선불전자지급수단 구매  충전금액, 무이자할부 이용거래, 포인트 사용거래 중 포인트금액, 지방세, ')
guide_tbl sample: (491, {'source': 'C:/SKN19/data-preprocessing/data/hyundai\\hyundai_giftcard.json'}, '}\n  },\n  {\n    "id": "환불ㆍ기부 안내_1",\n    "title": "신청 방법 및 시간",\n    "content": "현대카드 홈페이지 및 고객센터를 통해 신청(계좌이체/대체입금/기부 신청 등의 형태로 환불 가능)\\n24시간 신청 가능\\n*평일 오후 6시 이후, 주말 및 공휴일에 환불 신청 시 다음 영업일에 처리",\n    "text"')
card_tbl metadata keys: [('title', 22

In [6]:
from pathlib import Path
import os
import sys
import psycopg2
from dotenv import load_dotenv

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
    raise FileNotFoundError("Could not find repo root with app/ directory")

ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)

def db_config():
    host = os.getenv("DB_HOST_IP") or os.getenv("DB_HOST")
    cfg = {
        "host": host,
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": int(os.getenv("DB_PORT", "0")) or None,
    }
    missing = [k for k, v in cfg.items() if not v]
    if missing:
        raise ValueError(f"Missing DB settings: {missing}")
    return cfg

def fetch(sql, params=None):
    with psycopg2.connect(**db_config()) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params or ())
            return cur.fetchall()

sql = """
SELECT id, metadata->>'id', metadata->>'card_name', metadata->>'category'
FROM card_tbl
WHERE metadata->>'id' = '서울시다둥이행복카드_11';
"""
print("card_tbl:", fetch(sql))

sql = """
SELECT id, metadata->>'id', metadata->>'card_name', metadata->>'category'
FROM guide_tbl
WHERE metadata->>'id' = '서울시다둥이행복카드_11';
"""
print("guide_tbl:", fetch(sql))


card_tbl: []
guide_tbl: []


In [9]:
from pathlib import Path
import os
import sys
import psycopg2
from dotenv import load_dotenv

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
    raise FileNotFoundError("Could not find repo root with app/ directory")

ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)

def db_config():
    host = os.getenv("DB_HOST_IP") or os.getenv("DB_HOST")
    cfg = {
        "host": host,
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": int(os.getenv("DB_PORT", "0")) or None,
    }
    missing = [k for k, v in cfg.items() if not v]
    if missing:
        raise ValueError(f"Missing DB settings: {missing}")
    return cfg

def fetch(sql, params=None):
    with psycopg2.connect(**db_config()) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params or ())
            return cur.fetchall()

terms = [ "서울시 다산콜센터 "]

def build_like_where(terms):
    clauses = []
    params = []
    for term in terms:
        clauses.append(
            "(content ILIKE %s OR metadata->>'title' ILIKE %s OR "
            "metadata->>'category' ILIKE %s OR metadata->>'category1' ILIKE %s OR "
            "metadata->>'category2' ILIKE %s)"
        )
        params.extend([f"%{term}%"] * 5)
    return " OR ".join(clauses), params

where_sql, params = build_like_where(terms)

sql_card = f"""
SELECT id, metadata->>'id', metadata->>'title', metadata->>'card_name',
       metadata->>'category', metadata->>'category1', metadata->>'category2',
       LEFT(content, 180)
FROM card_tbl
WHERE {where_sql}
ORDER BY id DESC
LIMIT 20;
"""

sql_guide = f"""
SELECT id, metadata->>'id', metadata->>'title', metadata->>'card_name',
       metadata->>'category', metadata->>'category1', metadata->>'category2',
       LEFT(content, 180)
FROM guide_tbl
WHERE {where_sql}
ORDER BY id DESC
LIMIT 20;
"""

print("card_tbl matches:")
print(fetch(sql_card, params))

print("guide_tbl matches:")
print(fetch(sql_guide, params))


card_tbl matches:
[(199, 'card_150', '발급 대상', None, None, 'N/A', 'N/A', '제목: 발급 대상\n\n내용: - 서울시 거주, 2자녀 이상, 막내 만 18세 이하 가구의 부모  확인 서류 : 주민등록등본 및 가족관계증명서  서울시 다산콜센터 : 120')]
guide_tbl matches:
[]


In [10]:
from pathlib import Path
import os
import sys
import json
import psycopg2
from dotenv import load_dotenv

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
    raise FileNotFoundError("Could not find repo root with app/ directory")

ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)

def db_config():
    host = os.getenv("DB_HOST_IP") or os.getenv("DB_HOST")
    cfg = {
        "host": host,
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": int(os.getenv("DB_PORT", "0")) or None,
    }
    missing = [k for k, v in cfg.items() if not v]
    if missing:
        raise ValueError(f"Missing DB settings: {missing}")
    return cfg

def fetch(sql, params=None):
    with psycopg2.connect(**db_config()) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params or ())
            return cur.fetchall()

sql = """
SELECT id, metadata
FROM card_tbl
WHERE metadata->>'title' = '발급 대상'
ORDER BY id DESC
LIMIT 5;
"""
rows = fetch(sql)

for row in rows:
    print("id:", row[0])
    print(json.dumps(row[1], ensure_ascii=False, indent=2))


id: 199
{
  "id": "card_150",
  "title": "발급 대상",
  "category1": "N/A",
  "category2": "N/A"
}


In [14]:
from pathlib import Path
import os
import sys
import json
import psycopg2
from dotenv import load_dotenv

def find_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / "app").exists():
            return p
    raise FileNotFoundError("Could not find repo root with app/ directory")

ROOT_DIR = find_repo_root(Path.cwd().resolve())
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

load_dotenv(ROOT_DIR / ".env", override=True)

def db_config():
    host = os.getenv("DB_HOST_IP") or os.getenv("DB_HOST")
    cfg = {
        "host": host,
        "dbname": os.getenv("DB_NAME"),
        "user": os.getenv("DB_USER"),
        "password": os.getenv("DB_PASSWORD"),
        "port": int(os.getenv("DB_PORT", "0")) or None,
    }
    missing = [k for k, v in cfg.items() if not v]
    if missing:
        raise ValueError(f"Missing DB settings: {missing}")
    return cfg

def fetch(sql, params=None):
    with psycopg2.connect(**db_config()) as conn:
        with conn.cursor() as cur:
            cur.execute(sql, params or ())
            return cur.fetchall()

# 1) 카드 테이블에서 특정 키워드 매칭 문서 metadata 전체 보기
keyword = "신용상태"

sql = """
SELECT id, metadata, LEFT(content, 120)
FROM card_tbl
WHERE content ILIKE %s OR metadata->>'title' ILIKE %s
ORDER BY id DESC
LIMIT 10;
"""
rows = fetch(sql, (f"%{keyword}%", f"%{keyword}%"))

for r in rows:
    print("id:", r[0])
    print("metadata:", json.dumps(r[1], ensure_ascii=False, indent=2))
    print("content:", r[2])
    print("-" * 40)

# 2) guide_tbl도 동일 확인
rows = fetch(sql.replace("card_tbl", "guide_tbl"), (f"%{keyword}%", f"%{keyword}%"))

for r in rows:
    print("guide id:", r[0])
    print("metadata:", json.dumps(r[1], ensure_ascii=False, indent=2))
    print("content:", r[2])
    print("-" * 40)


guide id: 122
metadata: {
  "id": "guide_101",
  "title": "금리인하요구권 안내",
  "category1": "금융안내",
  "category2": "카드대출 예약신청"
}
content: 제목: 금리인하요구권 안내

내용: 장기카드대출 약정 후 신용등급 또는 개인신용평점 개선 등으로 회원님의 신용상태가 현저히 변경된 경우, 금리인하를 요구할 수 있는 금리인하요구권제도를 운영 중입니다.
신청 대상: 신
----------------------------------------
guide id: 115
metadata: {
  "id": "guide_96",
  "title": "금리인하요구권 안내",
  "category1": "금융안내",
  "category2": "우량대출"
}
content: 제목: 금리인하요구권 안내

내용: 장기카드대출 약정 후 신용등급 또는 개인신용평점 개선 등으로 회원님의 신용상태가 현저히 변경된 경우, 금리인하를 요구할 수 있는 금리인하요구권제도를 운영 중이니 이용에 참고하시기 
----------------------------------------
guide id: 109
metadata: {
  "id": "guide_92",
  "title": "금리인하요구권 안내",
  "category1": "금융안내",
  "category2": "카드대금 납부"
}
content: 제목: 금리인하요구권 안내

내용: 단기카드대출, 장기카드대출 약정 후 신용등급 또는 개인신용평점 개선 등으로 회원님의 신용상태가 현저히 변경된 경우, 금리인하를 요구할 수 있는 금리인하요구권 제도를 운영 중이니 이
----------------------------------------
guide id: 101
metadata: {
  "id": "guide_86",
  "title": "금리인하요구권 안내",
  "category1": "금융안내",
  "category2": "현금서비스"
}
conte